In [1]:
import pandas as pd
import glob
import ntpath
import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)


In [2]:
def readxl(filename):
    '''
    This reads the excel sheet counts when in the correct format
    
    '''
    counts = pd.read_excel(filename, sheet_name= 'Counts', header=2)
    movements = pd.read_excel(filename, sheet_name= 'Layout', usecols= 'M:O', nrows=12)
    counts = counts.merge(movements, left_on='turn', right_on='movement code')
    counts['intersection'] = ntpath.basename(filename).split('.')[0]
    return counts

def reformat(df):
    df = df.melt(id_vars= ['intersection','approach','direction','start','end'], 
             value_vars=['motorcycle','passenger car','two-axle truck','multi-axle truck','minibus taxi','standard bus'],
             var_name='vehicle class', value_name='Count')

    #df['start'], df['end'] =  pd.to_datetime(df['start']), pd.to_datetime(df['end'])

    df15 = df.groupby(['intersection','vehicle class','approach','direction','end']).sum()
    df60 = df15.rolling(4).sum()
    
    return df60


In [3]:
path ='D:/001_Projects/01 - GIPTN/01 - Microsimulation/01 - Beach Road Complete/Traffic Counts/raw/' 

file_list = glob.glob(path + "/*.xlsx")
df_list = [readxl(file) for file in file_list]

if df_list:
    counts = pd.concat(df_list, axis=0, sort=False) 


In [7]:
df60f.head()

vehicle class            minibus taxi  motorcycle  multi-axle truck  \
intersection   end                                                    
Beach & Beukes 06:15:00           1.0         4.0               8.0   
               06:30:00           2.0         2.0               6.0   
               06:45:00           3.0         3.0               3.0   
               07:00:00           7.0         1.0               2.0   
               07:15:00           7.0         3.0               4.0   

vehicle class            passenger car  standard bus  two-axle truck  
intersection   end                                                    
Beach & Beukes 06:15:00          351.0          21.0            15.0  
               06:30:00          275.0          16.0            16.0  
               06:45:00          253.0          13.0            14.0  
               07:00:00          271.0          10.0            13.0  
               07:15:00          360.0          12.0            18.0

In [8]:
filename = 'grouped2.csv'
reformat(counts).to_csv(path+filename)

In [4]:
def plot_total(vehicle_class):
    data = [go.Bar(x=df60f.index, y=df60f[vehicle_class])]

    layout = go.Layout(
        title='60 Min Count Data',
        xaxis=dict(
            title='Time Ending',
            titlefont=dict(
                family='Courier New, monospace',
                size=14,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='Volume (60 mins)',
            titlefont=dict(
                family='Courier New, monospace',
                size=14,
                color='#7f7f7f'
            )
        )
    )

    fig = go.Figure(data=data, layout=layout)

    return plotly.offline.iplot(fig, filename='jupyter-basic_bar')

def plot_int(intersection, vehicle_class):
    data = [go.Bar(x=df60f.xs(intersection).index, y=df60f.xs(intersection)[vehicle_class])]

    layout = go.Layout(
        title='60 Min Count Data',
        xaxis=dict(
            title='Time Ending',
            titlefont=dict(
                family='Courier New, monospace',
                size=14,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='Volume (60 mins)',
            titlefont=dict(
                family='Courier New, monospace',
                size=14,
                color='#7f7f7f'
            )
        )
    )

    fig = go.Figure(data=data, layout=layout)

    return plotly.offline.iplot(fig, filename='jupyter-basic_bar')


In [5]:
df60f = reformat(counts).groupby(['end','vehicle class']).Count.sum().unstack()
plot_total('passenger car')

In [6]:
df60f = reformat(counts).groupby(['intersection','end','vehicle class']).Count.sum().unstack()
plot_int('Beach & Beukes', 'passenger car')

In [11]:

filename = 'PeriodTotal.csv'
path ='D:/001_Projects/01 - GIPTN/01 - Microsimulation/01 - Beach Road Complete/Traffic Counts/%s' % (filename)
df60.groupby(['end']).Count.sum().reset_index().to_csv(path)